In [1]:
# Crea una tabla de cambios en nivel del mar

# Importamos librerías
import pandas as pd
import xarray as xr


In [2]:
# Carpetas
path   = "../../Bases_de_datos/Sea_level_rise/ar6-regional-confidence/"
f_name = [ "location_list.lst", "regional/confidence_output_files/" 
    + "medium_confidence/ssp245/total_ssp245_medium_confidence_values.nc" ]

# Cargamos la lista de mareógrafos
df = pd.read_table( path + f_name[0], index_col = "locations",
    names = ["name", "locations", "lat", "lon"] ).loc[1:2359]

df

,name,lat,lon
locations,,,
1,BREST,48.38,-4.49
2,SWINOUJSCIE,53.92,14.23
3,SHEERNESS,51.45,0.74
5,HOLYHEAD,53.31,-4.62
7,CUXHAVEN_2,53.87,8.72
...,...,...,...
2326,MAYPORT,30.40,-81.43
2329,REDWOOD_CITY,37.51,-122.21
2330,PORT_CHICAGO,38.05,-122.04


In [3]:
# Cargamos los niveles del mar
ds = xr.open_dataset( path + f_name[1] ).sel( quantiles = 0.5, years = 2050 )

# Seleccionamos los mareógrafos
ds_1 = ds.sel( locations = slice(1, 2538) )

# Seleccionamos los datos en malla y los guardamos como NetCDF
ds.sel( locations = ds["locations"][ ~ds["locations"].isin(
    ds_1["locations"] ) ] ).to_dataframe().reset_index().set_index(
    ["lat", "lon"] ).drop( columns = ["years", "quantiles", "locations"]
    ).to_xarray().to_netcdf(
    "../results/Climate/sea_level_change_ssp245_2050.nc" )

PermissionError: [Errno 13] Permission denied: '/Users/rodrigo/Desktop/UNESCO Climate Transition/UNESCO_climate/results/Climate/sea_level_change_ssp245_2050.nc'

In [ ]:
# Asignamos el aumento de nivel del mar a los mareógrafos
df["Sea level change [mm]"] = ds_1.to_dataframe(
    ).drop( columns = ["quantiles", "years"] )["sea_level_change"]

# Guardamos el archivo
df.to_csv("../share/Climate/sea_level_change.csv")

df

,name,lat,lon,Sea level change [mm]
locations,,,,
1,BREST,48.38,-4.49,211.0
2,SWINOUJSCIE,53.92,14.23,247.0
3,SHEERNESS,51.45,0.74,250.0
5,HOLYHEAD,53.31,-4.62,220.0
7,CUXHAVEN_2,53.87,8.72,266.0
...,...,...,...,...
2326,MAYPORT,30.40,-81.43,304.0
2329,REDWOOD_CITY,37.51,-122.21,203.0
2330,PORT_CHICAGO,38.05,-122.04,202.0
